In [2]:
import shots_data_retriever
from shots_data_retriever import ShotsDataRetriever
import importlib
import warnings
import pandas as pd

# Filter dtype warnings. Column 10 can contain str, int, and float types 
# which spams a warning in the output
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
importlib.reload(shots_data_retriever)

shotsDataRetriever = ShotsDataRetriever()
df = shotsDataRetriever.get_all_shots()

Retrieving shots data: 100%|██████████| 40/40 [01:36<00:00,  2.41s/it]


In [3]:
print(df.columns)

Index(['game_id', 'period', 'time_in', 'time_remaining', 'team_id', 'is_goal',
       'x_coord', 'y_coord', 'shooter_name', 'goalie_name', 'shot_type',
       'is_empty_net', 'situation_type'],
      dtype='object')


In [4]:
# Average shot rate
single_game_time = 1 # in hours, assumption from google doc
total_games = df['game_id'].nunique()
total_game_time = single_game_time * total_games
shot_location_league = df.groupby(['x_coord', 'y_coord']).size().reset_index(name='shot_count')
shot_location_league['shot_rate'] = shot_location_league['shot_count'] / total_game_time
# shot_location_league.to_csv('test_league.csv', index=False)

In [5]:
# Shot rate by team
total_game_time_per_team = df.groupby('team_id')['game_id'].nunique().reset_index(name='time_played')
shot_location_group = df.groupby(['team_id', 'x_coord', 'y_coord']).size().reset_index(name='shot_count')
shot_location_group = pd.merge(shot_location_group, total_game_time_per_team, on='team_id', how='left')
shot_location_group['shot_rate'] = shot_location_group['shot_count'] / shot_location_group['time_played']
# shot_location_group.to_csv('test_team.csv', index=False)

In [76]:
# Difference from league average
full_df = pd.DataFrame()
d = pd.DataFrame()
d['team_id'] = shot_location_group['team_id']
d['x_coord'] = shot_location_group['x_coord']
d['y_coord'] = shot_location_group['y_coord']

d = d.groupby('team_id')
for g in d:
    team_id = g[1]['team_id'].iloc[0]
    x_coord = g[1]['x_coord'].iloc[0]
    y_coord = g[1]['y_coord'].iloc[0]
    merge_df = pd.merge(
        shot_location_group,
        shot_location_league[['x_coord', 'y_coord', 'shot_rate']].rename(columns={'shot_rate': 'shot_rate_league'}),
        on=['x_coord', 'y_coord'],
        how='left'
    )
    merge_df = merge_df.drop('shot_rate_league_y', axis=1)
    full_df = pd.concat([full_df, shot_location_group[shot_location_group['team_id'] == team_id]])
    full_df = pd.merge(full_df, merge_df, how='left')
    full_df = full_df.drop('shot_rate_league_x', axis=1)
    full_df = full_df.drop('shot_count', axis=1)
    full_df = full_df.drop('time_played', axis=1)
print(full_df)
full_df.to_csv('full_df.csv', index=False)

        team_id  x_coord  y_coord  shot_rate  shot_rate_league
0             1      0.0    -26.0   0.001577          0.000293
1             1      0.0    -18.0   0.001577          0.000391
2             1      0.0      2.0   0.001577          0.000781
3             1      0.0      3.0   0.001577          0.000976
4             1      0.0     11.0   0.001577          0.000098
...         ...      ...      ...        ...               ...
145439       30     97.0     22.0   0.001548          0.000488
145440       30     97.0     23.0   0.001548          0.000195
145441       30     98.0    -20.0   0.001548          0.000195
145442       30     99.0     -0.0   0.001548          0.000098
145443       30     99.0     11.0   0.001548          0.000293

[145444 rows x 5 columns]


In [ ]:
# Temporary test

import plotly.express as px

# Create the shot map
fig = px.density_heatmap(team_df, x='x_coord', y='y_coord', z='is_goal', 
                          title=f'Shot Map for team 15', 
                          labels={'x_coord': 'X Coordinate', 'y_coord': 'Y Coordinate'},
                          color_continuous_scale='Viridis')

# Save as HTML
fig.write_html('team_shot_map.html')

In [ ]:
# Temporary test

import matplotlib.pyplot as plt

def plot_shot_map(df, team_name):
    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(10, 5))

    # Create the rink layout (you may need to customize this)
    rink_width = 100
    rink_height = 85
    ax.set_xlim(-rink_width/2, rink_width/2)
    ax.set_ylim(-rink_height/2, rink_height/2)

    # Plot shot coordinates
    shot_x = df['x_coord']
    shot_y = df['y_coord']
    ax.scatter(shot_x, shot_y, color='red', alpha=0.5, label='Shots')

    # Labels and title
    ax.set_title(f'Shot Map for {team_name}')
    ax.set_xlabel('Width of Rink (ft)')
    ax.set_ylabel('Height of Rink (ft)')
    ax.legend()
    plt.show()

# Call the function with your filtered DataFrame and team name
plot_shot_map(team_df, "Team 15")  # Re